In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import (Input, Dense, GlobalAveragePooling1D, Conv1D, 
                                     AveragePooling1D, UpSampling1D, Flatten)

import numpy as np
import pandas as pd

from datetime import datetime
import os
import sys
import pickle
import time
import argparse
from tqdm import tqdm

sys.path.insert(0, '/scratch/nj594/xai/helpers')
from evaluate import evaluate

# IMPORTANT: SET RANDOM SEEDS FOR REPRODUCIBILITY
os.environ['PYTHONHASHSEED'] = str(420)
import random
random.seed(420)
np.random.seed(420)
tf.random.set_seed(420)


# Get Arguments
batch_size = 64


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#Set Model Dir
method = 'gradcam'
model_dir = 'gradcam'
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

# Load Data

data_dir = os.path.join(os.getcwd(), 'data')

X_train = np.load(os.path.join(data_dir, 'X_train.npy'), allow_pickle=True)
X_val = np.load(os.path.join(data_dir, 'X_val.npy'), allow_pickle=True)
X_test = np.load(os.path.join(data_dir, 'X_test.npy'), allow_pickle=True)

y_train = np.load(os.path.join(data_dir, 'y_train.npy'), allow_pickle=True)
y_val = np.load(os.path.join(data_dir, 'y_val.npy'), allow_pickle=True)
y_test = np.load(os.path.join(data_dir, 'y_test.npy'), allow_pickle=True)

preds_train = np.load(os.path.join(data_dir, 'predictions_train.npy'), allow_pickle=True)
preds_discrete_train = np.eye(2)[preds_train.argmax(1)]
preds_val = np.load(os.path.join(data_dir, 'predictions_val.npy'), allow_pickle=True)
preds_discrete_val = np.eye(2)[preds_val.argmax(1)]
preds = np.load(os.path.join(data_dir, 'predictions.npy'), allow_pickle=True)
preds_discrete = np.eye(2)[preds.argmax(1)]


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

# Load Model

params = {
    #NN Hyperparameters
    "input_shape": [1000, 1],
    "num_categories": 2,
    "conv_subsample_lengths": [1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2],
    "conv_filter_length": 8,
    "conv_num_filters_start": 32,
    "conv_init": "he_normal",
    "conv_activation": "relu",
    "conv_dropout": 0.2,
    "conv_num_skip": 2,
    "conv_increase_channels_at": 4,
    "compile": False,
    "is_regular_conv": False,
    "is_by_time": False,
    "is_by_lead": False,
    "ecg_out_size": 64,
    "nn_layer_sizes" : None,
    "is_multiply_layer": False, 
}

#Stanford Model
sys.path.insert(0, '/scratch/nj594/ecg/models/stanford')
import network

model_input = Input(shape=(1000,1))

cnn = network.build_network(**params) 
cnn = Model(cnn.inputs, cnn.layers[-4].output)

net = cnn(model_input)
net = GlobalAveragePooling1D()(net)
out = Dense(2, activation='softmax')(net)

model = Model(model_input, out)
model.summary()

# Model Checkpointing
save_dir = 'model'
model_dir = os.path.join(os.getcwd(), save_dir)
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)
model_weights_path = os.path.join(model_dir, 'model_weights.h5')

# Get Checkpointed Model
print(model_weights_path)
model.load_weights(model_weights_path)
model.trainable = False

OPTIMIZER = tf.keras.optimizers.Adam(1e-3)
METRICS = [ 
  tf.keras.metrics.AUC(name='auroc'),
  tf.keras.metrics.AUC(curve='PR', name='auprc'),
  tf.keras.metrics.TopKCategoricalAccuracy(k=1, name='accuracy'),
]

model.compile(
    loss='categorical_crossentropy',
    optimizer=OPTIMIZER,
    metrics=METRICS,
)

2022-04-16 19:23:23.715251: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64:/.singularity.d/libs
2022-04-16 19:23:23.715274: W tensorflow/stream_executor/cuda/cuda_driver.cc:312] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-16 19:23:23.715290: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cs042.hpc.nyu.edu): /proc/driver/nvidia/version does not exist
2022-04-16 19:23:23.715470: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-16 19

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1000, 1)]         0         
_________________________________________________________________
functional_1 (Functional)    (None, 4, 256)            5246112   
_________________________________________________________________
global_average_pooling1d (Gl (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 514       
Total params: 5,246,626
Trainable params: 5,238,882
Non-trainable params: 7,744
_________________________________________________________________
/scratch/nj594/xai/ecg/experiments/model/model_weights.h5


# Grad CAM

In [39]:
superpixel_size = 8

#### Get Model for Feature Map ####
submodel_index, submodel = 1, model.layers[1]
x = submodel.outputs[0]
for layer_index in range(submodel_index+1, len(model.layers)):
    extracted_layer = model.layers[layer_index]
    print('Surgically appending layer : '+str(extracted_layer))
    x = extracted_layer(x)
flat_model = Model(inputs=submodel.inputs, outputs=x)

grad_model = tf.keras.models.Model(
    flat_model.inputs, [flat_model.layers[67].get_output_at(0), flat_model.output]
)

Surgically appending layer : <tensorflow.python.keras.layers.pooling.GlobalAveragePooling1D object at 0x7fcc3c029c10>
Surgically appending layer : <tensorflow.python.keras.layers.core.Dense object at 0x7fcc3c0345d0>


In [93]:
def ponderate_output(output, grad):
    """
    Perform the ponderation of filters output with respect to average of gradients values.
    """
    weights = tf.reduce_mean(grad, 0)

    # Perform ponderated sum : w_i * output[:, i]
    cam = tf.reduce_sum(tf.multiply(weights, output), axis=-1)

    return cam

explanations = []
for y_class in range(y_test.shape[1]):
    cams = []
    for i in tqdm(range(44)): #mini-batch
        with tf.GradientTape() as tape:
            if (i+1)*50 > len(X_test):
                end = len(X_test)+1
            else:
                end = (i+1)*50
            inputs = tf.cast(X_test[i*50:end], tf.float32)
            tape.watch(inputs)
            conv_outputs, predictions = grad_model(inputs)
            loss = predictions[:, y_class]

        grads = tape.gradient(loss, conv_outputs)
        grads = (
             tf.cast(conv_outputs > 0, "float32")
             * tf.cast(grads > 0, "float32")
             * grads
         )
        
        
        cam = UpSampling1D(size=superpixel_size)(
            tf.expand_dims(tf.stack([ponderate_output(output, grad) for output, grad in zip(conv_outputs, grads)]), -1)
        )
#         cam = tf.maximum(cam, 0)
        cams.append(cam)

    cams = tf.concat(cams, 0).numpy()
    explanations.append(cams)

explaining_time = time.time() - t

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [01:05<00:00,  1.50s/it]


In [94]:
with open(os.path.join(method, 'explanations.pkl'), 'wb') as f:
    pickle.dump(explanations, f)

with open(os.path.join(method, 'explaining_time.pkl'), 'wb') as f:
    pickle.dump(explaining_time, f) 

# Evaluation

In [95]:
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

# Load Evaluator Model

eval_dir = os.path.join(os.getcwd(), 'evaluation', 'evaluator-data')
evaluator_model = tf.keras.models.load_model(os.path.join(eval_dir, 'surrogate.h5'))

OPTIMIZER = tf.keras.optimizers.Adam(1e-3)
METRICS = [ 
  tf.keras.metrics.AUC(name='auroc'),
  tf.keras.metrics.AUC(curve='PR', name='auprc'),
  tf.keras.metrics.TopKCategoricalAccuracy(k=1, name='accuracy'),
]

evaluator_model.compile(
    loss='categorical_crossentropy',
    optimizer=OPTIMIZER,
    metrics=METRICS,
)

In [114]:
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#### Retrospective Evaluation ####

# Exclusion
retro_ex_test = evaluate(X_test, explanations, evaluator_model, y_test, y_test, 
                         mode = 'exclude', method = method)

# Inclusion
retro_in_test = evaluate(X_test, explanations, evaluator_model, y_test, y_test, 
                         mode = 'include', method = method)

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#### Prospective Evaluation ####

# Exclusion
pro_ex_test = evaluate(X_test, explanations, evaluator_model, preds_discrete, y_test, 
                         mode = 'exclude', method = method)

# Inclusion
pro_in_test = evaluate(X_test, explanations, evaluator_model, preds_discrete, y_test, 
                         mode = 'include', method = method)

100
99
95
90
85
75
50
25
15
10
5
1
0


In [120]:
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# Combine Results
tags = ['retro_ex','retro_in','pro_ex','pro_in']
result_list = [retro_ex_test,retro_in_test,pro_ex_test,pro_in_test]

results = {}
for res, tag  in zip(result_list, tags):
    res = {k+'-'+tag:v for k,v in res.items()}
    results = {**results, **res}

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

# Save

### Create Results Dictionary
header = ["model_dir", "explaining_time"]
metrics = ['AUC_acc','AUC_auroc','AUC_log_likelihood','AUC_log_odds']
for tag in tags:
    header += [x+'-'+tag for x in metrics]
    
results['model_dir'] = model_dir
results["explaining_time"] = explaining_time
results = {k:v for k,v in results.items() if k in header}

### Convert to DataFrame
results_df = pd.DataFrame(results, index=[0])
results_df = results_df[header]

In [122]:
### Append DataFrame to csv
results_path = method+'/results.csv'
if os.path.exists(results_path):
    results_df.to_csv(results_path, mode='a',  header=False)
else:
    results_df.to_csv(results_path, mode='w',  header=True)